<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/S_VS_T_XGB_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
!pip install bert-for-tf2

     |████████████████████████████████| 1.2MB 4.3MB/s 
     |████████████████████████████████| 51kB 3.4MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=1776b3404539fdb3afe5b7afa66bb0e2cfe36e981c7816804fb77e0bf47794b6
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=69138135989ea2c2977a484b827f6732668978b9f354878e70fc52e62e89bdcc
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=e7bdcae2fa0e0d478afa5eca74675119d99ee6367cb869eaf8b67d604f55561a
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os, re
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
os.environ['KAGGLE_KEY'] = "001b3a30170775e55950edb6ff0c9b17"
!kaggle datasets download -d nikhileswarkomati/suicide-watch

 84% 97.0M/115M [00:01<00:00, 61.3MB/s]
100% 115M/115M [00:01<00:00, 85.3MB/s] 


In [ ]:
!unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
  inflating: SuicideAndDepression_Detection.csv  


In [43]:
data = pd.read_csv('/content/SuicideAndDepression_Detection.csv', lineterminator = '\n')
data.sample(5)

,text,class
210166,"Open letter to Minecraft Dear Minecraft,\n\nI ...",teenagers
78018,"Done.I am so tired of waking up, i wake up in ...",SuicideWatch
291067,"No prospects, bleak future. Dead End. 21 years...",SuicideWatch
119428,Let's draw together https://r7.whiteboardfox.c...,teenagers
171558,I just turned 15 today. Is there any advice th...,teenagers


In [ ]:
print(data.shape)
data['class'].value_counts()

(348111, 2)


teenagers       116037
SuicideWatch    116037
depression      116037
Name: class, dtype: int64

In [ ]:
def preprocess(string):
    phrase = str(string)
    phrase = re.sub('[^a-z]+', ' ', phrase, flags = re.IGNORECASE)
    phrase = re.sub('(\s+)', ' ', phrase)
    phrase = re.sub('http\S+', ' ', phrase)
    phrase = phrase.lower()
    words_li = ['filler']
    li = list(stopwords.words()) + words_li
    text_tokens = word_tokenize(phrase)
    return " ".join([word for word in text_tokens if word not in li])
print(data['text'].apply(lambda x: len(x.split(' '))).sum())

59527144


In [ ]:
data['text'] = data['text'].map(lambda string: preprocess(string))

In [ ]:
print(data['text'].apply(lambda x: len(x.split(' '))).sum())

59527144


In [ ]:
data.loc[(data['class'] != 'depression'), 'text'][:1000].shape

(1000,)

In [44]:
# X = data.iloc[:, 0].values
# y = data.iloc[:, 1].values

# le = LabelEncoder()

# y = le.fit_transform(y)

# train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
# train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
# print(train_X.shape, test_y.shape, val_X.shape)

#------------------------------------------------------------------------------------------
X = data.loc[(data['class'] != 'depression'), 'text'].values
y = data.loc[(data['class'] != 'depression'), 'class'].values

le = LabelEncoder()

y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
print(train_X.shape, test_y.shape, val_X.shape)

(23207,) (116037,) (92830,)


In [ ]:
[preproele for ele in data.loc[(data['class'] != 'depression'), 'text'][:10000].values]

In [16]:
embeddings = embed(data.loc[(data['class'] != 'depression'), 'text'][:10000].values)
train_y = le.fit_transform(data.loc[(data['class'] != 'depression'), 'class'][:10000].values)
test_embeddings = embed(data.loc[(data['class'] != 'depression'), 'text'][10000:15000].values)
test_y = le.transform(data.loc[(data['class'] != 'depression'), 'class'][10000:15000].values)


# ------------------------------------------------------------------------------------
# embeddings = embed([preprocess(ele) for ele in data.loc[(data['class'] != 'depression'), 'text'][:10000].values])
# train_y = le.fit_transform(data.loc[(data['class'] != 'depression'), 'class'][:10000].values)
# test_embeddings = embed([preprocess(ele) for ele in data.loc[(data['class'] != 'depression'), 'text'][10000:15000].values])
# test_y = le.transform(data.loc[(data['class'] != 'depression'), 'class'][10000:15000].values)

In [45]:
xgb1 = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])

xgb1.fit(train_X, train_y)
y_pred = xgb1.predict(test_X)

print('accuracy %s' % accuracy_score(y_pred, test_y))
print(classification_report(test_y, y_pred))

accuracy 0.8852176460956419
              precision    recall  f1-score   support

           0       0.92      0.85      0.88     58019
           1       0.86      0.92      0.89     58018

    accuracy                           0.89    116037
   macro avg       0.89      0.89      0.89    116037
weighted avg       0.89      0.89      0.89    116037



In [46]:
from sklearn.linear_model import SGDClassifier

sgd2 = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier()),
              ])

sgd2.fit(train_X, train_y)
y_pred = sgd2.predict(test_X)

print('accuracy %s' % accuracy_score(y_pred, test_y))
print(classification_report(test_y, y_pred))

accuracy 0.9277385661470048
              precision    recall  f1-score   support

           0       0.94      0.92      0.93     58019
           1       0.92      0.94      0.93     58018

    accuracy                           0.93    116037
   macro avg       0.93      0.93      0.93    116037
weighted avg       0.93      0.93      0.93    116037

